In [ ]:
# # # #
# Elliptic Curve Diffie-Hellman (ECDH) Key Exchange
# Ergin Sezgin
# 02/10/2024
#

##  AES  Encryption

import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from binascii import hexlify, unhexlify

# Generate AES Key and IV
key = os.urandom(32)  # AES-256 key
iv = os.urandom(16)   # AES block size IV

key_hex = hexlify(key).decode('utf-8')
iv_hex = hexlify(iv).decode('utf-8')

print(f"Key generated (Hexadecimal):\n{key_hex}")
print(f"\nInitialization vector generated (Hexadecimal):\n{iv_hex}")

# Data to be encrypted
plain_text = "This is a test data."
print(f"\nData to be encrypted:\n{plain_text}")
plain_bytes = plain_text.encode('utf-8')

# Encrypt the data
cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
encryptor = cipher.encryptor()
cipher_bytes = encryptor.update(plain_bytes) + encryptor.finalize()
cipher_text_hex = hexlify(cipher_bytes).decode('utf-8')

print(f"\nData encrypted (Hexadecimal):\n{cipher_text_hex}")

# Decrypt the data
decryptor = cipher.decryptor()
decrypted_bytes = decryptor.update(cipher_bytes) + decryptor.finalize()
decrypted_text = decrypted_bytes.decode('utf-8')

print(f"\nData decrypted:\n{decrypted_text}")


In [ ]:
## PBE_Encryption

import hashlib
import base64

password = 'RandomPassword'
salt = 'random_salt'.encode('utf-8')

# Derive the key using PBKDF2 with 10000 iterations and a 32-byte key length
derived_key = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 10000, dklen=32)

# Convert the derived key to Base64
derived_key_base64 = base64.b64encode(derived_key).decode('utf-8')

print(f"Password: {password}")
print(f"Derived Key: {derived_key_base64}")


In [ ]:
## RSA Encyrption

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from binascii import hexlify

# Generate RSA Key Pair
key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Get the private key and public key in PEM format
private_key = key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
).decode('utf-8')

public_key = key.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).decode('utf-8')

print(f"Public Key:\n{public_key}")
print(f"\nPrivate Key:\n{private_key}")

# Data to be encrypted
plain_text = "This is a test data."
print(f"\nData to be encrypted:\n{plain_text}")
plain_bytes = plain_text.encode('utf-8')

# Encrypt the data using the public key
cipher_bytes = key.public_key().encrypt(
    plain_bytes,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

cipher_text_hex = hexlify(cipher_bytes).decode('utf-8')
print(f"\nData encrypted (Hexadecimal):\n{cipher_text_hex}")

# Decrypt the data using the private key
decrypted_bytes = key.decrypt(
    cipher_bytes,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

decrypted_text = decrypted_bytes.decode('utf-8')
print(f"\nData decrypted:\n{decrypted_text}")


In [ ]:
## Elliptic Curve Diffie-Hellman (ECDH) Key Exchange

from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# Generate ECDH key pairs for both parties
first_party_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
second_party_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())

# Public keys for both parties
first_party_public_key = first_party_private_key.public_key()
second_party_public_key = second_party_private_key.public_key()

# Derive shared secrets
first_party_shared_secret_with_second = first_party_private_key.exchange(ec.ECDH(), second_party_public_key)
second_party_shared_secret_with_first = second_party_private_key.exchange(ec.ECDH(), first_party_public_key)

# Check if shared secrets match
shared_secret_matches = first_party_shared_secret_with_second == second_party_shared_secret_with_first

if shared_secret_matches:
    print('Shared secret matches between parties.')
else:
    print("Shared secret doesn't match.")
